## Data Colletion from Twitter

In [3]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import numpy as np
import re
import csv
from collections import Counter
import nltk

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [4]:
from datetime import datetime, timedelta

def date_range(start, end):
    start_date = datetime.strptime(start, '%Y-%m-%d').date()
    end_date = datetime.strptime(end, '%Y-%m-%d').date()
    delta = end_date - start_date 
    days = [start_date + timedelta(days=i) for i in range(delta.days + 1)]
    return list(map(lambda n: n.strftime("%Y-%m-%d"), days))

data_collection_period = date_range('2022-01-01', '2022-09-01')

In [5]:
stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]

def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp


def get_most_recent_tweets(city, amount, distance = '20km'):
    df_city = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance}').get_items(), amount))[['date', 'rawContent']]
    df_city['cleaned_content'] = [clean_tweet(i) for i in df_city['rawContent']]
    df_city.to_csv(f"data/{city}/{amount}_most_recent_tweets.csv")
    return df_city


def get_tweets_over_period(city, amount_per_day, data_collection_period, distance='20km'):
    df_city = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance} since:{data_collection_period[0]} until:{data_collection_period[1]} ').get_items(), amount_per_day))[['date', 'rawContent']]
    for i in range(2, len(data_collection_period)-1):
        df_temp = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance} since:{data_collection_period[i]} until:{data_collection_period[i+1]} ').get_items(), amount_per_day))[['date', 'rawContent']]
        df_city = df_city.append(df_temp)
    df_city.to_csv(f"data/{city}/{amount_per_day*len(data_collection_period)}_tweets_over_period.csv")
    return df_city


In [6]:
%%time
get_most_recent_tweets("Singapore", 3000, distance='20km')
get_tweets_over_period("Singapore", 100, data_collection_period)

KeyboardInterrupt: 

## Exploratory Data Analysis

In [9]:
stopwords = nltk.corpus.stopwords.words('english')
english_words_set = set(nltk.corpus.words.words())

# # sanity test for some english words
# list_english_words = list(english_words_set )
# print("sustainability" in list_english_words)

In [10]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [11]:
import csv

def write_to_csv(file_name, list_words):
    with open(f"{file_name}.csv", 'w', encoding='UTF8', newline='\n') as f:
        writer = csv.writer(f)
        writer.writerow(["word", "frequency"])
        for tup in list_words:
            writer.writerow(list(tup))

In [12]:
def generate_word_corpus(filename):
    corpus = []
    with open(filename) as file:
        csvreader = csv.reader(file)
        next(csvreader)
        for row in csvreader:
            if len(row) > 3:
                all_words_in_sentence = row[3].strip().split(" ")
                corpus += all_words_in_sentence
    return corpus


def generate_word_frequency(city, approach, corpus):
    counter=Counter(corpus)
    most=counter.most_common()
    write_to_csv(f"data/{city}/{approach}_topMostFrequentWords", most[:100])
    write_to_csv(f"data/{city}/{approach}_topLeastFrequentWords", most[-100::])
    
    # non-English words
    non_english_words = []
    for tup in most:
        if (lemmatizer.lemmatize(tup[0]) not in english_words_set and tup[0].isnumeric() == False):
            non_english_words.append(tup)
    write_to_csv(f"data/{city}/{approach}_nonEnglishWords", non_english_words)

    
def average_sentence_length(city):
    total_length = 0
    count = 0
    with open(f"data/{city}.csv") as file:
        csvreader = csv.reader(file)
        next(csvreader)
        for row in csvreader:
            if len(row) > 3:
                all_words_in_sentence = row[3].split(" ")
                total_length += len(all_words_in_sentence)
                count += 1
    print("Average sentence length:", total_length/count)

In [15]:
import os
def create_relative_english_frequency_col(df):
    def calculate_relative_frequency(content):
        english_word_bools = [lemmatizer.lemmatize(word) in english_words_set for word in str(content).split() if word.isalpha()]
        if not english_word_bools:
            return 0
        return english_word_bools.count(True)/len(english_word_bools) 
    df["english_relative_frequency"] = df.cleaned_content.apply(calculate_relative_frequency)
    return df

def filter_by_english_thresholds(min_thresh, max_thresh, original_dataset, create_file = False):
    output_dir = os.path.splitext(original_dataset)[0]
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    with open(original_dataset, "r") as infile:
        source_df = pd.read_csv(infile)
        out_df = create_relative_english_frequency_col(source_df)
        out_df = out_df[(min_thresh <= out_df["english_relative_frequency"]) & (out_df["english_relative_frequency"] < max_thresh)]
    
    if create_file: 
        with open(output_dir + f"/{min_thresh}_to_{max_thresh}_english_words.csv", "w") as outfile:
            outfile.write(out_df.to_csv())
            
    return out_df


In [16]:
for i in range(10):
    filter_by_english_thresholds(i/10, (i+1)/10, "data/Singapore/GenerateMostRecentTweets/3000_most_recent_tweets.csv", create_file = True)
filter_by_english_thresholds(1, 1.01, "data/Singapore/GenerateMostRecentTweets/3000_most_recent_tweets.csv")

,Unnamed: 0,date,rawContent,cleaned_content,english_relative_frequency
0,0,2022-09-26 13:21:07+00:00,@A_Magic_Forest @nft_finley @tinalautrup Beautiful,beautiful,1.0
2,2,2022-09-26 13:21:05+00:00,"@olex_scherba @nytimes And he is Xi’s as well, shame on him!",he is xi s as well shame him,1.0
3,3,2022-09-26 13:21:04+00:00,@CHIEtizensOFC Vitality\n\n#ChieFilomeno,vitality,1.0
4,4,2022-09-26 13:21:04+00:00,@NithyaMenenOffI Hi,hi,1.0
5,5,2022-09-26 13:21:03+00:00,@MFA_China Liar,liar,1.0
...,...,...,...,...,...
2989,2989,2022-09-26 12:36:47+00:00,Фото из очереди на пограничный переход в Казахстан – стоят часами без движения https://t.co/TJa5pUeviJ,NaN,1.0
2992,2992,2022-09-26 12:36:44+00:00,😌😌 https://t.co/JPvVSFK7v3,NaN,1.0
2994,2994,2022-09-26 12:36:44+00:00,@Agent_Tangerine @CarlosV71486200 @_biasedlfc_lad @SazMCFC @goal Am I right? I think they only watched football 2018 onwards.,am i right i think they only watched football 2018 onwards,1.0
2996,2996,2022-09-26 12:36:43+00:00,@bvbyshinyyx WHEN AND WHERE??,when where,1.0


In [66]:
filter_by_english_thresholds(0, 0.1, "data/Singapore/GenerateMostRecentTweets/3000_most_recent_tweets.csv")

,Unnamed: 0,date,rawContent,cleaned_content,english_relative_frequency
8,8,2022-09-26 13:21:00+00:00,Yasss,yasss,0.000000
15,15,2022-09-26 13:20:53+00:00,HBD https://t.co/Jw8o7AE4Mw,hbd,0.000000
20,20,2022-09-26 13:20:44+00:00,pls stfu oml 😂😂😂😂😂😂😂,pls stfu oml,0.000000
23,23,2022-09-26 13:20:43+00:00,夕飯を茹で卵とプロテインでいけるかと思ったけど、3時間でお腹空いてきちゃった🥲はてどうしよう〜,3,0.000000
29,29,2022-09-26 13:20:36+00:00,@nas_zaidi Hahaha,hahaha,0.000000
37,37,2022-09-26 13:20:30+00:00,@Rekha51627292 @anbarasu_Boss @Kavin_m_0431 @KavinFansClub Unmai Dhaaney. Eeee😁🏃🏻 https://t.co/LRiejzWD3a,unmai dhaaney eeee,0.000000
50,50,2022-09-26 13:20:21+00:00,先週久しぶりにヨーロッパに行って、周りの友人たちからの言動からESGについて真面目に考えてて、変化を感じた。正直経済成長している最中の国では優先順位は低いと思う,esg,0.000000
58,58,2022-09-26 13:20:12+00:00,@ineorino @FWBESS Beban &amp; tuntutan,beban amp tuntutan,0.000000
66,66,2022-09-26 13:20:01+00:00,新加坡武装部队今年同澳大利亚国防部队人员举行三叉戟演习，两国一共派出超过1600名部队人员参与，新加坡海军也首次出动两艘海军登陆舰。 https://t.co/bAaPuQrjJX,1600,0.000000
68,68,2022-09-26 13:20:00+00:00,@sekaijii :DD,dd,0.000000


In [88]:
# generate 3000 most recent tweets & analysis
singlish_most_recent_tweets_corpus = generate_word_corpus("data/Singapore/GenerateMostRecentTweets/3000_most_recent_tweets.csv")
generate_word_frequency("Singapore", "GenerateMostRecentTweets/3000_most_recent_tweets", singlish_most_recent_tweets_corpus)

In [83]:
# run once, generate cleaned tweets separately for 2nd approach to reduce wait time
df = pd.read_csv('data/Singapore/GenerateTweetsOverAPeriod/24400_tweets_over_period.csv', index_col=0)
df['cleaned_content'] = [clean_tweet(i) for i in df['rawContent']]
df.to_csv(f"data/Singapore/GenerateTweetsOverAPeriod/24400_cleaned_tweets_over_period.csv")

In [89]:
# generate analysis for tweets over period 2022-01-01 and 2022-09-01
singlish_over_period_tweets_corpus = generate_word_corpus(f"data/Singapore/GenerateTweetsOverAPeriod/{100*len(data_collection_period)}_cleaned_tweets_over_period.csv")
generate_word_frequency("Singapore", f"GenerateTweetsOverAPeriod/{100*len(data_collection_period)}_cleaned_tweets_over_period", singlish_over_period_tweets_corpus)